<a href="https://colab.research.google.com/github/ZonaQuant/Notebooks/blob/main/Public_GetP123RSData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Portfolio123 Parameters
ApiKey = 'xxxxxxxxxxxxxxxxxxxxxxx'
ApiId = 'yy'

RankingSystemID = 000000 # Portfolio123 Ranking System ID
Universe = 'zzzzzz'  
StartDate = '20200715' # YYYYMMDD
EndDate = '20201001' # YYYYMMDD
Freq = 'M' # Frequency of update M: Monthly, W-SAT: Weekly on Saturday

# Google Drive locations
GoogleDrive = '/content/gdrive/My Drive/Colab Notebooks/Modules/' # directory URL
GoogleDriveDataFile ='AIProjectData3.csv' # Name of file data is saved to


In [ ]:
#%%capture

# python libraries and options
import numpy as np
from numpy import random
import pandas as pd
import math
pd.set_option('display.max_columns', 7)
from sklearn.metrics import r2_score
from scipy import stats
import matplotlib.pyplot as plt

# Set p Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append(GoogleDrive)

# install Portfolio123 API
!pip install p123api
from p123api import Client, ClientException

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
DateListObj = pd.date_range(start=StartDate, end=EndDate, freq=Freq)
DateList = DateListObj.strftime("%Y-%m-%d").values
MaxAttempts = 10
AttemptCount = 0
DataFileValid = False
for RunDate in DateList:
  if AttemptCount < MaxAttempts:
    print(RunDate)
    client = Client(api_id=ApiId, api_key=ApiKey)
    ValidData = False
  while (AttemptCount < MaxAttempts) and (ValidData == False):
    try:
      RankData = client.rank_ranks({"pitMethod": "Complete",'rankingSystem': RankingSystemID, 'asOfDt': RunDate, 'universe': Universe, "nodeDetails": 'factor'})
      ValidData = True
    except ClientException as e:
      AttemptCount += 1
      print("p123api fail count: ", AttemptCount)
      print(e)
  if ValidData == True:
    DataFileValid = True
    RankDate = RankData['dt']
    tickers = RankData['tickers']
    Nodes = RankData['nodes']
    NodeNames = Nodes['names']
    NodeRanks = Nodes['ranks']
    Datafile=pd.DataFrame(data=NodeRanks, columns=NodeNames)
    del Datafile[NodeNames[0]]
    Datafile.insert(0, "Date", RunDate, True) 
    Datafile.insert(1, "Ticker", tickers, True)
    if RunDate != DateList[0]:
      result = result.append(Datafile, ignore_index=True, sort=False)
    else:
      result = Datafile
if DataFileValid == True:
  print(result)
  result.to_csv(GoogleDrive+GoogleDriveDataFile)



2020-07-31
2020-08-31
2020-09-30
            Date Ticker  Input1  ...  Input9  Random       Target
0     2020-07-31   COUP   90.09  ...   29.53   70.03 -100000000.0
1     2020-07-31   TWLO   96.33  ...   78.55    4.52 -100000000.0
2     2020-07-31    RNG   91.20  ...   50.68   92.42 -100000000.0
3     2020-07-31   SHOP   99.39  ...   17.21   25.71 -100000000.0
4     2020-07-31   CLDR   48.03  ...   70.79   13.60 -100000000.0
...          ...    ...     ...  ...     ...     ...          ...
1083  2020-09-30   DSPG    8.19  ...   38.94    9.56 -100000000.0
1084  2020-09-30   AUTO    5.62  ...   70.40   80.61 -100000000.0
1085  2020-09-30   SYNC    4.34  ...   76.12    5.88 -100000000.0
1086  2020-09-30  NINOY    7.92  ...   37.03   12.32 -100000000.0
1087  2020-09-30   IMAX   14.08  ...   61.92   11.29 -100000000.0

[1088 rows x 13 columns]
